In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets.widgets import Layout, Label, VBox, HBox, Dropdown, Button
from xgboost import XGBClassifier

import warnings
warnings.simplefilter(action='ignore')
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'xgboost'

# Recomendação de Esquema de tratamento de Malária usando IA

---
### Esquemas de tratamento de malária
Em uma análise de dados realizada no banco de dados SIVEP-Malária, foi identificado que o Esquema de Tratamento mais utilizado (82,8%) é o esquema **"Infecções pelo P. vivax, ou P. ovale com Cloroquina em 3 dias e Primaquina em 7 dias"** (aqui denominado **Esquema 11**). Os demais 17,2% dos indivíduos da base, foram tratados com algum outro esquema de tratamento. Atualmente existem 27 Esquemas de Tratamentos pré-definidos, além de uma opção "Outros Esquemas de Tratamentos".

In [ ]:
file = open("images/pizza.png", "rb")
image = file.read()
img1 = widgets.Image(value=image, format='png', width='25%')
widgets.HBox([img1])

### Modelo de IA
Utilizando Algoritmos de Aprendizado de Máquina e dados SIVEP-Malária, foi criado um Sistema de IA para recomendação de esquema de tratamento de malária. Considerando a grande quantidade de indivíduos tratados com o **Esquema 11**, desenvolvemos um modelo de aprendizado de máquina que calcula a probalidade de um indivíduo (identificado conforme dados do formulário abaixo), receber indicação para o **Esquema 11**, ou não, e neste cas0, o indivíduo deve receber indicação de um esquema de tratamento, diferente do **Esquema 11**.

### Como modelo toma decisão

Para realizar a recomendação, o modelo baseia-se em reconhimento de padrões que podem ser observados nos dados utilizados. A Figura 1 abaixo mostra a importância de cada variável para o modelo, ou seja, as variáveis mais relevantes utilizadas pelo modelo para a recomendação são nesta ordem: 1) "Resultado do Exame", 2) "Idade do paciente", e assim sucessivamente. A Figua 2 apresenta métricas de avaliação do modelo.

In [ ]:
file = open("images/ROC-Malaria.png", "rb")
image = file.read()
img1 = widgets.Image(value=image, format='png', width='45%')

file = open("images/SHAP-Malaria.jpeg", "rb")
image = file.read()
img2 = widgets.Image(value=image, format='png', width='45%')

widgets.HBox([img2, img1])


---

# Simulação

**Defina o perfil do paciente no formulário abaixo para obter recomendação se o Esquema de "Tratamento 11" é recomendado, ou se é recomendado "Outros Esquemas".**

In [ ]:
# Carrega valores dos options para as caixas de escolha de variáveis
def show_categories(col):
    df_categories = pd.read_csv('./dados/'+ col + '.csv')
    return df_categories

In [ ]:
# Dicionario de dados para nome das variáveis
sivep = {'tipo_lam':'Lâmina',
         'id_pacie':'Idade',
         'id_dimea':'em',
         'sexo':'Sexo',
         'raca':'Raça/Cor',
         'sintomas':'Sintomas',
         'cod_ocup':'Ocupação',
         'res_exam':'Res. exame',
         'qtd_cruz':'Qtd cruzes',
         'hemoparasi':'Hemoparasitas',
         'exame':'Tipo do exame',
         'gestante':'Gestante',
         'vivax':'Trat. vivax',
         'falciparum':'Trat. falciparum',
         'niv_esco':'Escolaridade',
         'mun_infe':'Município de infecção',
         'loc_infe':'Local de infecção'
        }

In [ ]:
import matplotlib.pyplot as plt

fields = []

def create_label(myArr):
    label = []
    for item in myArr:
        converted_list = [str(element) for element in item]
        label.append(converted_list)
    return label

def createField(colname, options):
    return Dropdown(options=options, description=sivep.get(colname))

# Executa o modelo de ML Carregado
def run_model(*ignore):
    #X = pd.read_pickle("./dados/X.pck").copy()

    X = pd.DataFrame(0, index=np.arange(1), columns=sivep.keys())

    # Preenche vetor de entrada com campos do formulario
    for f in fields[:-1]:
        # Retorna o nome da coluna baseado na descrição
        col = [var for var, desc in sivep.items() if desc == f.description]
        X.at[0,col] = f.value

    ############################

    y_pred = malaria_model.predict(X)
    sim = 'Considerando os dados informados o "Esquema de Tratamento 11" recomendado.'
    nao = 'Considerando os dados informados é recomendado "Outros Esquemas".'

    y_proba =  malaria_model.predict_proba(X)

    #fig = plt.figure()
    #ax = fig.add_axes([0,0,1,1])
    #ax.barh(['Esquema 11', 'Outros Esquemas'], y_proba[0], height=0.1)
    #plt.savefig("./images/result.png")

    resultado.value = "Recomendação: \n >> Classe: " + str(y_pred[0]) + "\n"
    resultado.value = resultado.value + "\n-- Probabilidade --"
    resultado.value = resultado.value + "\nOutros Esquemas de Tratamento (Classe 0): " + str(round(100*y_proba[0][0],2)) + '%\n'
    resultado.value = resultado.value + "Esquema de Tratamento 11 (Classe 1): " + str(round(100*y_proba[0][1],2)) + '%\n'

    #resultado.value = resultado.value + str(y_pred[0]) #.replace("1", sim).replace("0", nao) ##+ "\n\n\n" + str(X) ##+ "\n" + str(malaria_model)

    ############################

# Criando campos do formulário
for col in ['tipo_lam', 'id_pacie', 'id_dimea', 'sexo', 'raca', 'sintomas', 'cod_ocup', 'res_exam', 'qtd_cruz',
            'hemoparasi', 'exame', 'gestante', 'vivax', 'falciparum', 'niv_esco']:
    fields.append(createField(col, create_label(show_categories(col).values)))

# variavel id_pacie
fields[1].options=np.arange(1,366)
fields[1].layout=Layout(width='140px')

# variavel id_dimea
fields[2].layout=Layout(width='140px')

# Município de Infecção
fields.append(Dropdown(options=[0,1], description='mun_infe'))

# Local de Infecção
fields.append(Dropdown(options=[0,1], description='loc_infe'))

button = Button(description='Executar')
fields.append(button)
button.on_click(run_model)

#----

sociodemo = VBox([Label("_____________________________________________________"),
                  Label("Informações Sociodemográficas"),
                  Label("_____________________________________________________"),

                  HBox([Label(value="Idade"), fields[1], fields[2]]),
                  fields[3], fields[4], fields[6], fields[11], fields[14]

                  #Label(value="Município de infecção"), fields[15],
                  #Label(value="Local de infecção"), fields[16]

                 ],
                  layout=Layout(width='40%', display='flex' , align_items='center'))

laboratorial = VBox([Label("_____________________________________________________"),
                     Label("Informações Epdemiológicas e Laboratóriais"),
                     Label("_____________________________________________________"),
                     fields[0], fields[5], fields[7], fields[8],
                     fields[9], fields[10], fields[12], fields[13],
                     Label(value=""),
                     Label("_____________________________________________________"),
                     button],
                     layout=Layout(width='40%', display='flex' , align_items='center'))

HBox([sociodemo, laboratorial])

In [ ]:
def get_bigger(args):
    resultado.rows = resultado.value.count('\n') + 1

resultado = widgets.Textarea(
    value='',
    description='Resultado',
    disabled=True,
    layout=widgets.Layout(width="90%")
)

resultado.observe(get_bigger, 'value')

widgets.HBox([resultado])


In [ ]:
#file = open("./images/result.png", "rb")
##image = file.read()
#img3 = widgets.Image(value=image, format='png')

#HBox([img3])

---

In [ ]:
# Loading model
print("Carregando modelo...")
malaria_model = XGBClassifier()
malaria_model.load_model("xgb_model.bin")
print("Pronto!")